### TF-IDF

In [ ]:
#   tf(term frequency) : 문서 내 단어 빈도
#   df(document frequency) : 문서마다 나오는 단어 빈도

# TF-IDF 점수가 높을수록 중요도가 높음

In [1]:
docs = [
  'python 데이터 python 프로그래밍',
  '데이터 분석',
  '빅 데이터 분석',
  '데이터 전처리 텍스트 전처리'
  ]

vocab=list() #단어사전 리스트

for doc in docs: #문서
    for w in doc.split(): #단어
        #         공백단위
        vocab.append(w) #단어 추가
      
vocab=list(set(vocab)) #중복 단어를 제거한 리스트
#           중복값 제거
vocab.sort() #오름차순 정렬
vocab

['python', '데이터', '분석', '빅', '전처리', '텍스트', '프로그래밍']

In [2]:
from math import log

N = len(docs) # 총 문서의 수

# tf(Term Frequency) : 단어의 빈도
# 특정 문서 d에서 특정 단어 t의 출현빈도
def tf(t, d):
    return d.count(t)

# df(Document Frequency) : 특정 단어가 등장한 문서의 수, 모든 문서에 자주 나오는 단어는 중요한 단어가 아닐 수 있음
#   논문에서 abstract, 서론, 본론, 결론, 참고문헌 항상 나오는 단어들 - 중요한 의미가 없음
# idf(Inverse Document Frequency) : df의 반비례하는 수
#   log( n / (1+df(t)))
#   희귀 단어의 경우 분모가 매우 작아지게 되어 idf 값이 너무 커지게 되므로 로그를 취함
#   한번도 등장하지 않는 단어라면 분모가 0이 되므로 1을 더하게 됨
def idf(t):     # df의 역수(단어가 몇개의 문서에 출현하는지)
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

#tf와 idf를 곱한 값
def tfidf(t, d):
    return tf(t,d)* idf(t)

In [3]:
import pandas as pd

result = []

for i in range(N): #문서 반복
    result.append([])
    d = docs[i]
    for j in range(len(vocab)): #단어 반복
        t = vocab[j]
        #1차원 배열의 끝에 추가
        result[-1].append(tf(t, d))
print(result)        

[[2, 1, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 2, 1, 0]]


In [4]:
df_tf = pd.DataFrame(result, columns = vocab)
df_tf

,python,데이터,분석,빅,전처리,텍스트,프로그래밍
0,2,1,0,0,0,0,1
1,0,1,1,0,0,0,0
2,0,1,1,1,0,0,0
3,0,1,0,0,2,1,0


In [5]:
#각 단어의 idf 계산

result = []

for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

df_idf = pd.DataFrame(result, index = vocab, columns = ["IDF"])
df_idf

,IDF
python,0.693147
데이터,-0.223144
분석,0.287682
빅,0.693147
전처리,0.693147
텍스트,0.693147
프로그래밍,0.693147


In [6]:
#모든 단어들의 tfidf 계산 - tfidf가 크면 중요도가 높고, 작으면 중요도가 낮음

result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))
        
df_tfidf = pd.DataFrame(result, columns = vocab)
df_tfidf

,python,데이터,분석,빅,전처리,텍스트,프로그래밍
0,1.386294,-0.223144,0.000000,0.000000,0.000000,0.000000,0.693147
1,0.000000,-0.223144,0.287682,0.000000,0.000000,0.000000,0.000000
2,0.000000,-0.223144,0.287682,0.693147,0.000000,0.000000,0.000000
3,0.000000,-0.223144,0.000000,0.000000,1.386294,0.693147,0.000000


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

#DTM(Document Term Matrix, 문서 단어 행렬)
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수 계산
print(vector.fit_transform(corpus).toarray())   # 출현빈도
print(vector.vocabulary_)   # 단어사전

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(corpus)

print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

#사이킷런의 TF-IDF 계산 방식이 약간 달라서 차이가 나지만 일반적으로 많이 사용하는 함수
#  tf-idf에 L2 정규화를 적용하여 값을 조정함

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
